HW02
Deep Learning, GIST RT5101-01, 2021, Spring, (Tue/Thurs 2:30~3:45)
***


How to submit your homework
Submit your jupyter notebook file with the filename of HW02_studentnumber.ipynb on GEL
Ex) HW02_20184021.ipynb

Submission deadline
2021.05.12, Wednesday 23:59 (PM)

Plagiarism
We encourage you to discuss this homework with your friends or TA, but you should write your own code.

### Problem 1. (Build simple fully connected layer model)
- step 1. Build your simple FCN layer by using fully connected layer
- step 2. train your own model
- Do not use coding lecture sample model connected

In [1]:
import torch
import torch.nn as nn

class SimpleFCN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleFCN, self).__init__()

        self.fc1 = nn.Linear(in_features=3*32*32, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=256)
        self.fc3 = nn.Linear(in_features=256, out_features=128)
        self.fc4 = nn.Linear(in_features=128, out_features=64)
        self.out_fc = nn.Linear(in_features=64, out_features=num_classes)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.relu(x)

        x = self.fc3(x)
        x = self.relu(x)

        x = self.fc4(x)
        x = self.relu(x)

        output = self.out_fc(x)

        return output


### Problem 2. (dataset code part)
- Load your dataset from dataset folder
- using rgb dataset ex) cifar10, cifar100
- you can use custom dataset from web site ex) dog vs cat

In [2]:
from torchvision import datasets, transforms

def dataset(is_train):

    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    dataset = datasets.CIFAR10(root='./dataset', train=is_train, transform=transform, download=True)

    return dataset

## Model train

In [3]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [4]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [5]:
GPU_NUM = 0
lr = 1e-4
batch_size = 64
epochs = 20

device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')


train_dataset = dataset(is_train=True)
val_dataset = dataset(is_train=False)


train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True)
                                                        
val_loader = torch.utils.data.DataLoader(val_dataset,
                                            batch_size=batch_size,
                                            shuffle=False)


model = SimpleFCN(num_classes=10).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr, momentum=0.9, weight_decay=5e-4)

criterion = nn.CrossEntropyLoss().to(device)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
train_loss_arr = []
train_acc_arr = []

val_loss_arr = []
val_acc_arr = []


for epoch in range(epochs):

    model.train()

    losses = AverageMeter()
    top1 = AverageMeter()

    for i, (data, target) in enumerate(train_loader):
        
        data = data.to(device)
        target = target.to(device)

        output = model(data) 

        loss = criterion(output, target)

        output.float()
        loss.float()

        prec1 = accuracy(output.data, target)
        prec1 = prec1[0]

        losses.update(loss.item(), data.size(0))
        top1.update(prec1.item(), data.size(0))

        if i % 100 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})'.format(epoch, i, len(train_loader), loss=losses))

        
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    train_loss_arr.append(losses.avg)
    train_acc_arr.append(top1.avg)
    print("train result: Loss: {}, Acc: {}\n".format(losses.avg, top1.avg))


    model.eval()
    with torch.no_grad():
        val_loss_sum = 0
        val_acc_sum = 0

        losses = AverageMeter()
        top1 = AverageMeter()

        for i, (data, target) in enumerate(val_loader):
            data = data.to(device)
            target = target.to(device)

            output = model(data) 

            loss = criterion(output, target)

            output.float()
            loss.float()

            prec1 = accuracy(output.data, target)

            prec1 = prec1[0]
            losses.update(loss.item(), data.size(0))
            top1.update(prec1.item(), data.size(0))

            if i % 100 == 0:
                print('Test: [{0}/{1}]\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(val_loader), loss=losses, top1=top1))

        val_loss_arr.append(losses.avg)
        val_acc_arr.append(top1.avg)
        print("Validation result: Loss: {}, Acc: {}\n".format(losses.avg, top1.avg))


Epoch: [0][0/782]	Loss 2.3281 (2.3281)
Epoch: [0][100/782]	Loss 2.3019 (2.3074)
Epoch: [0][200/782]	Loss 2.2896 (2.3066)
Epoch: [0][300/782]	Loss 2.2953 (2.3066)
Epoch: [0][400/782]	Loss 2.3095 (2.3065)
Epoch: [0][500/782]	Loss 2.2977 (2.3070)
Epoch: [0][600/782]	Loss 2.3233 (2.3065)
Epoch: [0][700/782]	Loss 2.3039 (2.3062)
train result: Loss: 2.3062437857818603, Acc: 10.198

Test: [0/157]	Loss 2.3023 (2.3023)	Prec@1 9.375 (9.375)
Test: [100/157]	Loss 2.3165 (2.3049)	Prec@1 9.375 (10.334)
Validation result: Loss: 2.304970682144165, Acc: 10.3

Epoch: [1][0/782]	Loss 2.3078 (2.3078)
Epoch: [1][100/782]	Loss 2.3057 (2.3057)
Epoch: [1][200/782]	Loss 2.3012 (2.3054)
Epoch: [1][300/782]	Loss 2.2961 (2.3050)
Epoch: [1][400/782]	Loss 2.2976 (2.3042)
Epoch: [1][500/782]	Loss 2.3319 (2.3040)
Epoch: [1][600/782]	Loss 2.2971 (2.3040)
Epoch: [1][700/782]	Loss 2.3092 (2.3039)
train result: Loss: 2.3037633891296387, Acc: 11.126

Test: [0/157]	Loss 2.2998 (2.2998)	Prec@1 12.500 (12.500)
Test: [100/157

### Problem 3. (Build CNN model with fc layer)
- step 1. Build your simple CNN layer by using convolutional layer and fully connected layer
- step 2. train your own model
- Do not use coding lecture sample model connected

In [7]:
import torch
import torch.nn as nn

class CNN_model(nn.Module):
    def __init__(self, num_classes):
        super(CNN_model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(5,5)) 
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5))
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=5*5*16, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)        
        self.out_fc = nn.Linear(in_features=84, out_features=num_classes)
        self.relu = nn.ReLU(inplace=True)        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)               

        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.relu(x)
        
        output = self.out_fc(x)        

        return output

### Problem 4. (Compare FCN model and CNN model)
- compare two model's performance
- subscribe your comment

In [8]:
GPU_NUM = 0
lr = 1e-4
batch_size = 64
epochs = 20

device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')

"""
train_dataset = dataset(is_train=True)
val_dataset = dataset(is_train=False)


train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True)
                                                        
val_loader = torch.utils.data.DataLoader(val_dataset,
                                            batch_size=batch_size,
                                            shuffle=False)
"""
model = CNN_model(num_classes=10).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss().to(device)

In [9]:
train_loss_arr = []
train_acc_arr = []

val_loss_arr = []
val_acc_arr = []


for epoch in range(epochs):

    model.train()

    losses = AverageMeter()
    top1 = AverageMeter()

    for i, (data, target) in enumerate(train_loader):
        
        data = data.to(device)
        target = target.to(device)

        output = model(data) 

        loss = criterion(output, target)

        output.float()
        loss.float()

        prec1 = accuracy(output.data, target)
        prec1 = prec1[0]

        losses.update(loss.item(), data.size(0))
        top1.update(prec1.item(), data.size(0))

        if i % 100 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})'.format(epoch, i, len(train_loader), loss=losses))

        
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    train_loss_arr.append(losses.avg)
    train_acc_arr.append(top1.avg)
    print("train result: Loss: {}, Acc: {}\n".format(losses.avg, top1.avg))


    model.eval()
    with torch.no_grad():
        val_loss_sum = 0
        val_acc_sum = 0

        losses = AverageMeter()
        top1 = AverageMeter()

        for i, (data, target) in enumerate(val_loader):
            data = data.to(device)
            target = target.to(device)

            output = model(data) 

            loss = criterion(output, target)

            output.float()
            loss.float()

            prec1 = accuracy(output.data, target)

            prec1 = prec1[0]
            losses.update(loss.item(), data.size(0))
            top1.update(prec1.item(), data.size(0))

            if i % 100 == 0:
                print('Test: [{0}/{1}]\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(val_loader), loss=losses, top1=top1))

        val_loss_arr.append(losses.avg)
        val_acc_arr.append(top1.avg)
        print("Validation result: Loss: {}, Acc: {}\n".format(losses.avg, top1.avg))

Epoch: [0][0/782]	Loss 2.3013 (2.3013)
Epoch: [0][100/782]	Loss 2.3063 (2.3036)
Epoch: [0][200/782]	Loss 2.2984 (2.3040)
Epoch: [0][300/782]	Loss 2.3019 (2.3037)
Epoch: [0][400/782]	Loss 2.2949 (2.3038)
Epoch: [0][500/782]	Loss 2.3004 (2.3037)
Epoch: [0][600/782]	Loss 2.3084 (2.3036)
Epoch: [0][700/782]	Loss 2.3033 (2.3034)
train result: Loss: 2.3035147527313233, Acc: 10.204

Test: [0/157]	Loss 2.3013 (2.3013)	Prec@1 12.500 (12.500)
Test: [100/157]	Loss 2.3068 (2.3031)	Prec@1 6.250 (9.994)
Validation result: Loss: 2.3029784851074218, Acc: 10.11

Epoch: [1][0/782]	Loss 2.3054 (2.3054)
Epoch: [1][100/782]	Loss 2.2970 (2.3023)
Epoch: [1][200/782]	Loss 2.3008 (2.3025)
Epoch: [1][300/782]	Loss 2.2990 (2.3026)
Epoch: [1][400/782]	Loss 2.3053 (2.3027)
Epoch: [1][500/782]	Loss 2.2978 (2.3026)
Epoch: [1][600/782]	Loss 2.2972 (2.3025)
Epoch: [1][700/782]	Loss 2.2978 (2.3025)
train result: Loss: 2.3024975106811523, Acc: 10.284

Test: [0/157]	Loss 2.3000 (2.3000)	Prec@1 12.500 (12.500)
Test: [100/

## FCN
Test Loss : 2.0315357999801638
Test Accuracy : 25.79

## CNN
Test Loss : 1.912327798652649
Test Accuracy : 30.76

I expected CNN models to perform significantly better than FCN models. But the actual results were slightly better. More research is needed on what methods should be used to achieve better results.